In [1]:
!pip install fiftyone
!pip install fiftyone-db-ubuntu2204
!pip install ultralytics

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.0/943.0 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
 

In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
import json
from collections import defaultdict
import matplotlib.pyplot as plt
from PIL import Image
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as F
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
from torchvision.transforms import functional as F

In [3]:
custom_dataset_dir = "/kaggle/working/dataset"
fo.config.dataset_zoo_dir = custom_dataset_dir

try:
    train_dataset = foz.load_zoo_dataset(
        "coco-2017",
        split="train",
        label_types=["detections", "segmentations"], 
        classes=["person"],  
        max_samples=3000, 
        dataset_name="coco-2017-train-persons", 
    )
    print("Train dataset loaded successfully!")
except Exception as e:
    print(f"Error loading train dataset: {e}")

try:
    val_dataset = foz.load_zoo_dataset(
        "coco-2017",
        split="validation",
        label_types=["detections", "segmentations"],
        classes=["person"],
        max_samples=750,
        dataset_name="coco-2017-validation-persons",
    )
    print("Validation dataset loaded successfully!")
except Exception as e:
    print(f"Error loading validation dataset: {e}")

 100% |██████|    1.9Gb/1.9Gb [6.0s elapsed, 0s remaining, 346.7Mb/s]       
Extracting annotations to '/kaggle/working/dataset/coco-2017/raw/instances_train2017.json'
 100% |████████████████| 3000/3000 [5.4m elapsed, 0s remaining, 9.2 images/s]       
Writing annotations for 3000 downloaded samples to '/kaggle/working/dataset/coco-2017/train/labels.json'
Dataset info written to '/kaggle/working/dataset/coco-2017/info.json'
Loading 'coco-2017' split 'train'
 100% |███████████████| 3000/3000 [1.0m elapsed, 0s remaining, 50.9 samples/s]      
Dataset 'coco-2017-train-persons' created
Train dataset loaded successfully!
Found annotations at '/kaggle/working/dataset/coco-2017/raw/instances_val2017.json'
 100% |██████████████████| 750/750 [1.4m elapsed, 0s remaining, 9.2 images/s]      
Writing annotations for 750 downloaded samples to '/kaggle/working/dataset/coco-2017/validation/labels.json'
Dataset info written to '/kaggle/working/dataset/coco-2017/info.json'
Loading 'coco-2017' split 'va

In [4]:
if len(train_dataset) == 0:
    print("Dataset is empty. Please check the dataset or filters.")
else:
    print(f"Dataset contains {len(train_dataset)} samples.")

if len(val_dataset) == 0:
    print("Dataset is empty. Please check the dataset or filters.")
else:
    print(f"Dataset contains {len(val_dataset)} samples.")

Dataset contains 3000 samples.
Dataset contains 750 samples.


In [5]:
class COCOParser:
    def __init__(self, anns_file, imgs_dir):
        self.imgs_dir = imgs_dir
        self.available_images = set(os.listdir(imgs_dir))  
        
        with open(anns_file, 'r') as f:
            coco = json.load(f)
        self.filename_to_id = {img['file_name']: img['id'] for img in coco['images']}
        self.id_to_filename = {v: k for k, v in self.filename_to_id.items()}  
        self.img_info = {img['id']: img for img in coco['images']}  
        self.annIm_dict = defaultdict(list)
        for ann in coco['annotations']:
            if ann['category_id'] == 1: 
                self.annIm_dict[ann['image_id']].append(ann)
        self.cat_dict = {cat['id']: cat for cat in coco['categories']}
    
    def get_available_image_ids(self):
        available_ids = []
        for filename in self.available_images:
            if filename in self.filename_to_id:
                available_ids.append(self.filename_to_id[filename])
        return available_ids
    
    def load_img(self, img_id):
        for filename, current_id in self.filename_to_id.items():
            if current_id == img_id:
                img_path = os.path.join(self.imgs_dir, filename)
                if os.path.exists(img_path):
                    return Image.open(img_path).convert("RGB")
        return None

class COCODataset(Dataset):
    def __init__(self, coco_parser, transforms=None):
        self.coco_parser = coco_parser
        self.transforms = transforms
        self.image_ids = coco_parser.get_available_image_ids()
    
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img = self.coco_parser.load_img(img_id)
        annotations = self.coco_parser.annIm_dict[img_id]

        boxes = []
        labels = []
        
        for ann in annotations:

            bbox = ann['bbox']
            x, y, w, h = bbox
            boxes.append([x, y, x + w, y + h])  

            labels.append(ann['category_id'])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
        }

        if self.transforms:
            img, target = self.transforms(img, target)
        
        return img, target

In [6]:
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class ToTensor:
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

class Normalize:
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, image, target):
        image = F.normalize(image, mean=self.mean, std=self.std)
        return image, target

class Resize:
    def __init__(self, size):
        self.size = size 

    def __call__(self, image, target):

        if not isinstance(image, Image.Image):
            raise TypeError("Resize harus diterapkan sebelum ToTensor!")
        
        original_size = image.size 
        image = F.resize(image, self.size)

        ratio_width = self.size[0] / original_size[0]  
        ratio_height = self.size[1] / original_size[1]  

        target["boxes"][:, [0, 2]] *= ratio_width
        target["boxes"][:, [1, 3]] *= ratio_height
        
        return image, target

def get_transform(train):
    transforms = []
    transforms.append(Resize((224, 224)))  
    transforms.append(ToTensor())         
    transforms.append(Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
    return Compose(transforms)

In [7]:
coco_annotations_file_train = "/kaggle/working/dataset/coco-2017/raw/instances_train2017.json"
coco_images_dir_train = "/kaggle/working/dataset/coco-2017/train/data"
coco_annotations_file_val = "/kaggle/working/dataset/coco-2017/raw/instances_val2017.json"
coco_images_dir_val = "/kaggle/working/dataset/coco-2017/validation/data"

coco_parser_train = COCOParser(coco_annotations_file_train, coco_images_dir_train)
coco_parser_val = COCOParser(coco_annotations_file_val, coco_images_dir_val)

In [8]:
import os
import shutil
from pathlib import Path

def convert_coco_to_yolo(coco_parser, output_dir, subset):
    images_dir = Path(output_dir) / "images" / subset
    labels_dir = Path(output_dir) / "labels" / subset
    images_dir.mkdir(parents=True, exist_ok=True)
    labels_dir.mkdir(parents=True, exist_ok=True)
    
    image_ids = coco_parser.get_available_image_ids()
    for img_id in image_ids:
        # Salin gambar ke direktori YOLO
        filename = coco_parser.id_to_filename[img_id]
        src_img_path = os.path.join(coco_parser.imgs_dir, filename)
        dst_img_path = images_dir / filename
        shutil.copy(src_img_path, dst_img_path)
        
        # Konversi anotasi
        annotations = coco_parser.annIm_dict.get(img_id, [])
        label_file = labels_dir / (filename.split('.')[0] + ".txt")
        with open(label_file, 'w') as f:
            for ann in annotations:
                img_info = coco_parser.img_info[img_id]
                img_w, img_h = img_info['width'], img_info['height']
                x, y, w, h = ann['bbox']
                
                # Konversi ke format YOLO
                x_center = (x + w / 2) / img_w
                y_center = (y + h / 2) / img_h
                width = w / img_w
                height = h / img_h
                
                # Kelas 0 karena hanya kelas 'person' yang digunakan
                f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

In [9]:
# Direktori output untuk dataset YOLO
output_dir = Path("/kaggle/working/yolo_dataset")

# Konversi dataset training dan validasi
convert_coco_to_yolo(coco_parser_train, output_dir, "train")
convert_coco_to_yolo(coco_parser_val, output_dir, "val")

# Buat file data.yaml
data_yaml_content = f"""
train: {str(output_dir / 'images/train')}
val: {str(output_dir / 'images/val')}
nc: 1
names: ['person']
"""

with open(output_dir / "data.yaml", "w") as f:
    f.write(data_yaml_content)

In [10]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # Pre-trained model weights untuk YOLOv8

model.train(
    data=str(output_dir / "data.yaml"),
    epochs=10,
    imgsz=640,
    batch=16,
    workers=16,
    device=0,
)

model.val()

model.export(format='onnx') 

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 129MB/s]


Ultralytics 8.3.80 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=16, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show

100%|██████████| 755k/755k [00:00<00:00, 24.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

val: Scanning /kaggle/working/yolo_dataset/labels/val.cache... 750 images, 0 backgrounds, 0 corrupt: 100%|██████████| 750/750 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:05<00:00,  7.95it/s]


                   all        750       3227      0.681       0.54       0.61      0.374


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.4ms preprocess, 2.7ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train2
Ultralytics 8.3.80 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.9 MB)
requirements: Ultralytics requirements ['onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 263.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 293.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 12.8s, installed 2 packages: ['onnxslim', 'onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX:

'runs/detect/train/weights/best.onnx'